# Pre-SynthSeg Processing

In [17]:
import os
import nibabel as nib
import numpy as np
import glob

In [87]:
pathname_main = 'E:/UKB'
fin_datasetfolder = 'E:/UKB/SynthSeg/test'
in_filePattern = os.path.join(fin_datasetfolder, '*_T1.nii.gz')
in_T1Files = glob.glob(in_filePattern)
ndata_in = len(in_T1Files)

outputdir = 'E:/UKB/SynthSeg/test'
SynthSegdir = 'D:/GitHub/SynthSeg'

In [88]:
in_T1Files

['E:/UKB/SynthSeg/test\\1000512_2_T1.nii.gz']

In [89]:
for file_path in in_T1Files:
    foldername, T1_name = os.path.split(file_path)
    print(foldername, T1_name)
    casename = T1_name.split('.')[0][:-3]
    print(casename)
    if os.path.exists(file_path):
        img = nib.load(file_path)
        data = img.get_fdata()
        voxel_dims = img.header.get_zooms()[:3]
        image_size = data.shape

        
        if not all(x % 32 == 0 for x in image_size):  # dimension not multiples of 32 
            pad_size = np.mod(image_size, 2)
            pad_size += np.mod(32 - np.mod(image_size + pad_size, 32), 32)
            data = np.pad(data, [(0, s) for s in pad_size], mode='constant')
        else:
            pad_size = [0, 0, 0]

        new_affine = img.affine.copy()
        new_affine[:3, :3] = np.diag([1.0, 1.0, 1.0])  # Set resolution to 1mm
        new_img = nib.Nifti1Image(data, affine=new_affine) 
        new_img_path = os.path.join(foldername, '1mmIntp', f'{casename}_1mmIntp.nii')
        os.makedirs(os.path.dirname(new_img_path), exist_ok=True)
        nib.save(new_img, new_img_path)

        # Check image dimensions after padding and interpolation
        if not np.allclose(new_img.header.get_zooms(), (1, 1, 1), atol=0.001):
            raise ValueError("Final interpolated image resolution is not [1 1 1]")



E:/UKB/SynthSeg/test 1000512_2_T1.nii.gz
1000512_2


# Post-SynthSeg Processing

In [2]:
import os
import glob
import numpy as np
import nibabel as nib
from scipy.ndimage import binary_dilation, generate_binary_structure

In [82]:
# Set up paths and initial conditions
pathname_main = 'E:/UKB'
fin_datasetfolder =  'E:/UKB/SynthSeg/test'
in_filePattern = os.path.join(fin_datasetfolder, '*_T1.nii.gz')
in_T1Files = glob.glob(in_filePattern)
ndata_in = len(in_T1Files)

outputdir = 'E:/UKB/SynthSeg/test'
SynthSegdir = 'D:/GitHub/SynthSeg'
flag_delete_intermediatefiles = False
flag_save_brain_mask = True


In [83]:
# Processing loop
for in_file in in_T1Files:
    foldername, T1_name = os.path.split(in_file)
    casename = T1_name[:-10]
    print(f'For {casename}')

    T1_full_path = os.path.join(foldername, T1_name)
    T1_1mmIntp_path = os.path.join(foldername, '1mmIntp', T1_name.replace('.nii.gz', '_1mmIntp.nii'))

    img = nib.load(T1_full_path)
    data = img.get_fdata()
    voxel_dims = img.header.get_zooms()
    size_t = data.shape

    if not all(x % 32 == 0 for x in size_t):  # dimension not multiples of 32
            pad_size = np.mod(size_t, 2)
            pad_size += np.mod(32 - np.mod(size_t + pad_size, 32), 32)
            cropped_orig_data = np.pad(data, [(0, s) for s in pad_size], mode='constant')
    else:
        pad_size = [0, 0, 0]

    seg_1mmIntp_path = os.path.join(outputdir, f'{casename}_1mmIntp_synthseg.nii')
    seg_img = nib.load(seg_1mmIntp_path)
    seg_data = seg_img.get_fdata()

    # Flip and adjust axes if needed
    flip_axes = [(np.isclose(seg_img.affine[i, i] / img.affine[i, i], -1, atol=0.0001), i) for i in range(3)]
    for flip, axis in flip_axes:
        if flip:
            seg_data = np.flip(seg_data, axis=axis)

    # Crop padding if added, like img_out = img_out(1:end-pad_size(1),1:end-pad_size(2),1:end-pad_size(3));
    seg_data = seg_data[:size_t[0], :size_t[1], :size_t[2]]

    # Prepare and save segmentation outputs
    mask = binary_dilation(seg_data > 0, structure=generate_binary_structure(3, 1))
    masked_data = data.copy()
    masked_data[~mask] = 0

    seg_name = os.path.join(outputdir, f'{casename}_T1_synthseg.nii')
    mask_path = seg_name.replace('.nii', '_brain_mask.nii')
    brain_path = seg_name.replace('.nii', '_brain.nii')

    # Create unique label map 0-99
    unique_labels = np.unique(seg_data)
    label_map = {label: idx for idx, label in enumerate(unique_labels)}
    mapped_seg_data = np.vectorize(label_map.get)(seg_data).astype(np.uint8)

    mapped_seg_data = mapped_seg_data.astype(np.uint8) # Ensure data type is uint8
    img.header.set_data_dtype(np.uint8) # Ensure data type is uint8

    nib.save(nib.Nifti1Image(mapped_seg_data, img.affine, img.header), seg_name)
    print('Seg in Nifti saved')
    #save in analysis format
    nib.analyze.save(nib.Nifti1Image(mapped_seg_data, img.affine, img.header), seg_name.replace('.nii', '.img'))
    print('Seg in analysis format saved')
    if flag_save_brain_mask:
        nib.save(nib.Nifti1Image(mask.astype(np.uint8), img.affine), mask_path)
        print('Mask in Nifti saved')
        #ensure the masked data is uint8
        masked_data_uint8 = (255 * (masked_data / masked_data.max())).astype(np.uint8)
        nib.save(nib.Nifti1Image(masked_data_uint8, img.affine), brain_path)
        print('Brain in Nifti saved')

    # Optional clean-up
    if flag_delete_intermediatefiles:
        os.remove(T1_1mmIntp_path)
        os.remove(seg_1mmIntp_path)


For 1000512_2
Seg in Nifti saved
Seg in analysis format saved
Mask in Nifti saved
Brain in Nifti saved


# debug

In [38]:
size_t

(168, 221, 182)

In [41]:
cropped_orig_data.shape

(192, 224, 192)

In [39]:
mask.shape

(144, 218, 172)

In [40]:
seg_data.shape

(144, 218, 172)

In [25]:
import numpy as np
import nibabel as nib

# Simulated values for size_t from an example image's shape
size_t = (91, 109, 91)

# Determine padding if dimensions are not multiples of 32
pad_size = [(0 if dim % 32 == 0 else 32 - dim % 32) for dim in size_t]
data = np.random.rand(*size_t)  # Simulated data array

# Apply padding
padded_data = np.pad(data, [(0, p) for p in pad_size], mode='constant')

print("Original size:", size_t)
print("Padding added:", pad_size)
print("New size:", padded_data.shape)


Original size: (91, 109, 91)
Padding added: [5, 19, 5]
New size: (96, 128, 96)


# only make unique labels for synthseg on original space

In [55]:
pathname_main = 'E:/UKB'
fin_datasetfolder = 'E:/UKB/test_gz_skull'
in_filePattern = os.path.join(fin_datasetfolder, '*_T1.nii.gz')
in_T1Files = glob.glob(in_filePattern)
ndata_in = len(in_T1Files)

outputdir = 'E:/UKB/SynthSeg/output_SynthSeg_parc_wo_resolution_conversion/'
SynthSegdir = 'D:/GitHub/SynthSeg'

flag_save_brain_mask = True
flag_save_brain_in_uint8 = False

In [58]:
in_T1Files

['E:/UKB/test_gz_skull\\1000512_2_T1.nii.gz',
 'E:/UKB/test_gz_skull\\1000709_2_T1.nii.gz',
 'E:/UKB/test_gz_skull\\1000797_2_T1.nii.gz',
 'E:/UKB/test_gz_skull\\1000835_2_T1.nii.gz',
 'E:/UKB/test_gz_skull\\1000878_2_T1.nii.gz',
 'E:/UKB/test_gz_skull\\1000957_2_T1.nii.gz',
 'E:/UKB/test_gz_skull\\1001036_2_T1.nii.gz',
 'E:/UKB/test_gz_skull\\1001320_2_T1.nii.gz',
 'E:/UKB/test_gz_skull\\1001613_2_T1.nii.gz',
 'E:/UKB/test_gz_skull\\1001632_2_T1.nii.gz',
 'E:/UKB/test_gz_skull\\1055837_2_T1.nii.gz']

In [21]:
# # test for one file
# img = nib.load(file_path)
# data = img.get_fdata()

# unique_labels = np.unique(data)
# label_map = {label: idx for idx, label in enumerate(unique_labels)}
# mapped_data = np.vectorize(label_map.get)(data).astype(np.uint8)
# img.header.set_data_dtype(np.uint8) # Ensure data type is uint8

# mapped_data = mapped_data.astype(np.uint8) # Ensure data type is uint8

# nib.save(nib.Nifti1Image(mapped_data, img.affine, img.header), file_path.replace('.nii.gz', '_mapped.nii'))
# nib.analyze.save(nib.Nifti1Image(mapped_data, img.affine, img.header), file_path.replace('.nii.gz', '_mapped.img'))

In [63]:
for T1file_path in in_T1Files:
    foldername, T1_name = os.path.split(T1file_path)
    # print(foldername, T1_name)
    casename = T1_name.split('.')[0].split('_T1')[0]
    print(f'For {casename}')

    T1img = nib.load(T1file_path)
    T1data = T1img.get_fdata()

    if T1file_path.endswith('.gz'):
        seg_name = os.path.join(outputdir, f'{casename}_T1_synthseg.nii.gz')
        print(seg_name)
    elif T1file_path.endswith('.nii'):
        seg_name = os.path.join(outputdir, f'{casename}_T1_synthseg.nii')
        print(seg_name)
    else:
        raise ValueError('Unknown file format')
    
    if os.path.exists(seg_name):
        seg_img = nib.load(seg_name)
        seg_data = seg_img.get_fdata()
    else:
        raise ValueError(f'{seg_name} does not exist')

    # Create unique label map 0-99
    unique_labels = np.unique(seg_data)
    label_map = {label: idx for idx, label in enumerate(unique_labels)}
    mapped_seg_data = np.vectorize(label_map.get)(seg_data).astype(np.uint8)

    T1img.header.set_data_dtype(np.uint8) # Ensure data type is uint8
    mapped_seg_data = mapped_seg_data.astype(np.uint8) # Ensure data type is uint8

    nib.save(nib.Nifti1Image(mapped_seg_data, seg_img.affine, T1img.header), os.path.join(outputdir, f'{casename}_T1_synthseg_relabeled98.nii'))
    print('Relabeled seg in Nifti saved')
    nib.analyze.save(nib.Nifti1Image(mapped_seg_data, seg_img.affine, T1img.header), os.path.join(outputdir, f'{casename}_T1_synthseg_relabeled98.img'))
    print('Relabeled seg in analysis format saved')

    if flag_save_brain_mask:
        mask_path = os.path.join(outputdir, f'{casename}_T1_brain_mask.nii')
        brain_path = os.path.join(outputdir, f'{casename}_T1_brain.nii')
        mask = binary_dilation(seg_data > 0, structure=generate_binary_structure(3, 1))
        masked_data = T1data.copy()
        masked_data[~mask] = 0
        nib.save(nib.Nifti1Image(mask.astype(np.uint8), T1img.affine), mask_path)
        print('Mask in Nifti saved')
        if flag_save_brain_in_uint8:
            masked_data = (255 * (masked_data / masked_data.max())).astype(np.uint8)
        # masked_data_uint8 = (255 * (masked_data / masked_data.max())).astype(np.uint8)
        nib.save(nib.Nifti1Image(masked_data, T1img.affine), brain_path)
        print('Brain in Nifti saved')


For 1000512_2
E:/UKB/SynthSeg/output_SynthSeg_parc_wo_resolution_conversion/1000512_2_T1_synthseg.nii.gz
Relabeled seg in Nifti saved
Relabeled seg in analysis format saved
Mask in Nifti saved
Brain in Nifti saved
For 1000709_2
E:/UKB/SynthSeg/output_SynthSeg_parc_wo_resolution_conversion/1000709_2_T1_synthseg.nii.gz
Relabeled seg in Nifti saved
Relabeled seg in analysis format saved
Mask in Nifti saved
Brain in Nifti saved
For 1000797_2
E:/UKB/SynthSeg/output_SynthSeg_parc_wo_resolution_conversion/1000797_2_T1_synthseg.nii.gz
Relabeled seg in Nifti saved
Relabeled seg in analysis format saved
Mask in Nifti saved
Brain in Nifti saved
For 1000835_2
E:/UKB/SynthSeg/output_SynthSeg_parc_wo_resolution_conversion/1000835_2_T1_synthseg.nii.gz
Relabeled seg in Nifti saved
Relabeled seg in analysis format saved
Mask in Nifti saved
Brain in Nifti saved
For 1000878_2
E:/UKB/SynthSeg/output_SynthSeg_parc_wo_resolution_conversion/1000878_2_T1_synthseg.nii.gz
Relabeled seg in Nifti saved
Relabeled 

In [73]:
import pandas as pd

In [78]:
# print the resolution of in_T1Files
df = pd.DataFrame(columns=['Case', 'Resolution'])
i = 0
for T1file_path in in_T1Files:
    
    foldername, T1_name = os.path.split(T1file_path)
    # print(foldername, T1_name)
    casename = T1_name.split('.')[0].split('_T1')[0]
    print(f'For {casename}')

    T1img = nib.load(T1file_path)
    T1data = T1img.get_fdata()
    voxel_dims = T1img.header.get_zooms()[:3]
    print(voxel_dims)
    df.loc[i] = [casename, voxel_dims]
    i += 1
df

For 1000512_2
(1.0000045, 1.0, 1.0)
For 1000709_2
(1.0000008, 1.0, 1.0)
For 1000797_2
(1.0000001, 1.0, 1.0)
For 1000835_2
(0.999996, 1.0, 1.0)
For 1000878_2
(0.9999995, 1.0, 1.0)
For 1000957_2
(1.0000004, 1.0, 1.0)
For 1001036_2
(1.0000036, 1.0, 1.0)
For 1001320_2
(1.0000001, 1.0, 1.0)
For 1001613_2
(0.99999815, 1.0, 1.0)
For 1001632_2
(1.0000021, 1.0, 1.0)
For 1055837_2
(1.0000033, 1.0, 1.0)


,Case,Resolution
0,1000512_2,"(1.0000045, 1.0, 1.0)"
1,1000709_2,"(1.0000008, 1.0, 1.0)"
2,1000797_2,"(1.0000001, 1.0, 1.0)"
3,1000835_2,"(0.999996, 1.0, 1.0)"
4,1000878_2,"(0.9999995, 1.0, 1.0)"
5,1000957_2,"(1.0000004, 1.0, 1.0)"
6,1001036_2,"(1.0000036, 1.0, 1.0)"
7,1001320_2,"(1.0000001, 1.0, 1.0)"
8,1001613_2,"(0.99999815, 1.0, 1.0)"
9,1001632_2,"(1.0000021, 1.0, 1.0)"
